# Plot Points from Streamlit Dashboard

This notebook loads and visualizes data points extracted from the Point Finder Dashboard.

**Supports both CSV and JSON formats**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from PIL import Image

# Set up matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Load Points from CSV

CSV format from the dashboard:
```
px_x,px_y,norm_x,norm_y
```

In [ ]:
# Load CSV file
csv_file = 'sample_points.csv'  # Change this to your downloaded CSV file

df = pd.read_csv(csv_file)
print(f"Loaded {len(df)} points from {csv_file}")
print("\nData preview:")
df.head()

## 2. Load Points from JSON (Alternative)

JSON format includes image metadata and is preferred for downstream work.

In [ ]:
# Load JSON file
json_file = 'sample_points.json'  # Change this to your downloaded JSON file

with open(json_file, 'r') as f:
    data = json.load(f)

print(f"Image: {data['image_path']}")
print(f"Dimensions: {data['width']} x {data['height']} px")
print(f"Rotation applied: {data['rotate_deg']}°")
print(f"Number of points: {len(data['points_px'])}")

# Convert to DataFrame for easier manipulation
df_json = pd.DataFrame({
    'px_x': [p[0] for p in data['points_px']],
    'px_y': [p[1] for p in data['points_px']],
    'norm_x': [p[0] for p in data['points_norm']],
    'norm_y': [p[1] for p in data['points_norm']]
})

print("\nJSON data as DataFrame:")
df_json.head()

## 3. Basic Plot: Points Only

Simple scatter plot of the extracted points.

In [ ]:
# Use whichever dataframe you loaded (CSV or JSON)
points_df = df  # or df_json

fig, ax = plt.subplots(figsize=(10, 8))

# Plot points
ax.scatter(points_df['px_x'], points_df['px_y'], 
           c='red', s=100, marker='o', 
           edgecolors='white', linewidths=2,
           label='Data Points', zorder=5)

# Add point numbers
for i, row in points_df.iterrows():
    ax.text(row['px_x'], row['px_y'] - 15, str(i+1),
           fontsize=10, color='red', weight='bold',
           ha='center', va='bottom')

ax.set_xlabel('X (pixels)')
ax.set_ylabel('Y (pixels)')
ax.set_title('Extracted Data Points')
ax.legend()
ax.grid(True, alpha=0.3)
ax.invert_yaxis()  # Match image coordinate system (Y increases downward)

plt.tight_layout()
plt.show()

## 4. Plot with Polynomial Fits

Fit polynomials to the points and visualize the curves.

In [ ]:
# Sort points by X coordinate for curve fitting
points_sorted = points_df.sort_values('px_x')

x = points_sorted['px_x'].values
y = points_sorted['px_y'].values

fig, ax = plt.subplots(figsize=(12, 8))

# Plot original points
ax.scatter(x, y, c='red', s=150, marker='o', 
           edgecolors='white', linewidths=2,
           label='Data Points', zorder=5)

# Fit and plot polynomials of different degrees
degrees = [2, 3, 4] if len(points_df) >= 5 else [2, 3]
colors = ['cyan', 'yellow', 'lime']
linestyles = ['-', '--', '-.']

x_smooth = np.linspace(x.min(), x.max(), 200)

for degree, color, ls in zip(degrees, colors, linestyles):
    if len(points_df) > degree:
        # Fit polynomial
        coeffs = np.polyfit(x, y, degree)
        poly = np.poly1d(coeffs)
        y_smooth = poly(x_smooth)
        
        # Plot curve
        ax.plot(x_smooth, y_smooth, 
               color=color, linewidth=2.5, 
               linestyle=ls, alpha=0.8,
               label=f'Polynomial (degree {degree})')

ax.set_xlabel('X (pixels)', fontsize=14)
ax.set_ylabel('Y (pixels)', fontsize=14)
ax.set_title('Data Points with Polynomial Fits', fontsize=16, weight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(True, alpha=0.3)
ax.invert_yaxis()  # Match image coordinate system

plt.tight_layout()
plt.show()

## 5. Overlay on Original Image (Optional)

If you have the original image, you can overlay the points and curves on it.

In [ ]:
# Load the original image (update path as needed)
image_path = 'test_image.png'  # Change to your image file

try:
    img = Image.open(image_path)
    
    # Apply rotation if specified in JSON
    if 'data' in locals() and data['rotate_deg'] != 0:
        img = img.rotate(data['rotate_deg'], expand=True, fillcolor='white')
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Display image
    ax.imshow(img, alpha=0.8)
    
    # Plot points
    ax.scatter(points_sorted['px_x'], points_sorted['px_y'], 
               c='red', s=150, marker='o', 
               edgecolors='white', linewidths=2,
               label='Extracted Points', zorder=5)
    
    # Add point numbers
    for i, (_, row) in enumerate(points_sorted.iterrows()):
        ax.text(row['px_x'], row['px_y'] - 20, str(i+1),
               fontsize=12, color='white', weight='bold',
               ha='center', va='bottom',
               bbox=dict(boxstyle='round,pad=0.4', 
                        facecolor='red', alpha=0.8))
    
    # Plot polynomial fits
    for degree, color, ls in zip(degrees, colors, linestyles):
        if len(points_df) > degree:
            coeffs = np.polyfit(x, y, degree)
            poly = np.poly1d(coeffs)
            y_smooth = poly(x_smooth)
            
            ax.plot(x_smooth, y_smooth, 
                   color=color, linewidth=3, 
                   linestyle=ls, alpha=0.9,
                   label=f'Fit (degree {degree})')
    
    ax.set_xlim(0, img.width)
    ax.set_ylim(img.height, 0)  # Invert Y
    ax.set_title('Points & Curves Overlaid on Original Image', 
                fontsize=16, weight='bold')
    ax.legend(loc='upper right', fontsize=12)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
except FileNotFoundError:
    print(f"Image file '{image_path}' not found. Skipping overlay plot.")
    print("Update the image_path variable to your image location.")

## 6. Export Polynomial Coefficients

Save the fitted polynomial coefficients for animation or further analysis.

In [ ]:
# Fit and save polynomial coefficients
poly_coeffs = {}

for degree in [2, 3, 4, 5]:
    if len(points_df) > degree:
        coeffs = np.polyfit(x, y, degree)
        poly_coeffs[f'degree_{degree}'] = coeffs.tolist()
        
        print(f"\nDegree {degree} polynomial:")
        print(f"  Coefficients: {coeffs}")
        
        # Create polynomial function
        poly = np.poly1d(coeffs)
        print(f"  Function: {poly}")

# Save to JSON for use in other scripts
output_file = 'polynomial_fits.json'
with open(output_file, 'w') as f:
    json.dump(poly_coeffs, f, indent=2)
    
print(f"\n✅ Polynomial coefficients saved to {output_file}")

## 7. Summary Statistics

In [ ]:
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(f"\nNumber of points: {len(points_df)}")
print(f"\nX-coordinate range: [{points_df['px_x'].min():.2f}, {points_df['px_x'].max():.2f}]")
print(f"Y-coordinate range: [{points_df['px_y'].min():.2f}, {points_df['px_y'].max():.2f}]")
print(f"\nX-coordinate mean: {points_df['px_x'].mean():.2f} ± {points_df['px_x'].std():.2f}")
print(f"Y-coordinate mean: {points_df['px_y'].mean():.2f} ± {points_df['px_y'].std():.2f}")

if 'data' in locals():
    print(f"\nImage dimensions: {data['width']} x {data['height']} px")
    print(f"Rotation applied: {data['rotate_deg']}°")

## Next Steps

1. **Replace sample files**: Update `csv_file`, `json_file`, and `image_path` with your actual downloaded files
2. **Adjust polynomial degrees**: Experiment with different polynomial degrees for best fit
3. **Use in Manim**: Import polynomial coefficients into Manim for animation
4. **Further analysis**: Apply curve fitting, interpolation, or other mathematical techniques

---

**Happy plotting! 📊✨**